In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision import models
import gc

In [13]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for MobileNet
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
dataset = datasets.ImageFolder(root=r"C:\Users\krish\Downloads\archive (6)\myData", transform=transform)

In [6]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f"Total images: {len(dataset)}, Train: {train_size}, Test: {test_size}")

Total images: 73139, Train: 58511, Test: 14628


In [8]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 43)
model = model.to(device)

C:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\krish/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

Epoch [1/10], Loss: 0.2753, Accuracy: 91.45%
Epoch [2/10], Loss: 0.1004, Accuracy: 96.85%
Epoch [3/10], Loss: 0.0750, Accuracy: 97.56%
Epoch [4/10], Loss: 0.0627, Accuracy: 98.07%
Epoch [5/10], Loss: 0.0552, Accuracy: 98.27%
Epoch [6/10], Loss: 0.0433, Accuracy: 98.63%
Epoch [7/10], Loss: 0.0430, Accuracy: 98.70%
Epoch [8/10], Loss: 0.0394, Accuracy: 98.82%
Epoch [9/10], Loss: 0.0310, Accuracy: 99.07%
Epoch [10/10], Loss: 0.0277, Accuracy: 99.13%


In [11]:
torch.save(model.state_dict(), "mobilenet_traffic_signs.pth")
print("Training complete. Model saved.")

Training complete. Model saved.


In [12]:
model.eval()  # Set to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_accuracy = 100 * correct / total
print(f"Final Test Accuracy: {test_accuracy:.2f}%")

Final Test Accuracy: 99.15%
